# This notebook is for geo data processing
geopandas has conflicts with anaconda, so install and use it in a virtual env called geoenv.

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point
from shapely import wkt
from shapely.geometry import *
import json

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline

In [2]:
fire_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/Fire_Incidents_SF.csv'
df_fire = pd.read_csv(fire_file)

C:\Users\Ruoying\anaconda3\envs\geoenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (10,12,13,20,38,45,46,55,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# fire data
df_fire.columns

Index(['Incident Number', 'Exposure Number', 'ID', 'Address', 'Incident Date',
       'Call Number', 'Alarm DtTm', 'Arrival DtTm', 'Close DtTm', 'City',
       'zipcode', 'Battalion', 'Station Area', 'Box', 'Suppression Units',
       'Suppression Personnel', 'EMS Units', 'EMS Personnel', 'Other Units',
       'Other Personnel', 'First Unit On Scene', 'Estimated Property Loss',
       'Estimated Contents Loss', 'Fire Fatalities', 'Fire Injuries',
       'Civilian Fatalities', 'Civilian Injuries', 'Number of Alarms',
       'Primary Situation', 'Mutual Aid', 'Action Taken Primary',
       'Action Taken Secondary', 'Action Taken Other',
       'Detector Alerted Occupants', 'Property Use', 'Area of Fire Origin',
       'Ignition Cause', 'Ignition Factor Primary',
       'Ignition Factor Secondary', 'Heat Source', 'Item First Ignited',
       'Human Factors Associated with Ignition', 'Structure Type',
       'Structure Status', 'Floor of Fire Origin', 'Fire Spread',
       'No Flame Spead'

In [4]:
# this is the primary situation: causes for fire. No null value here. very complete data
df_fire['Primary Situation'].nunique()

359

In [10]:
df_fire['Primary Situation'].unique()

array(['412 - Gas leak (natural gas or LPG)', '552 - Police matter',
       '210 - Steam Rupture, steam, other', '522 - Water or steam leak',
       '520 - Water problem, other',
       '733 - Smoke detector activation/malfunction',
       '711 - Municipal alarm system, Street Box False',
       '113 - Cooking fire, confined to container',
       '743 - Smoke detector, no fire, accidental',
       '551 - Assist PD or other Govern. Agency',
       '411 - Gasoline or other flammable liquid spill',
       '150 - Outside rubbish fire, other',
       '413 - Oil or other combustible liquid spill',
       '745 - Alarm system sounded/no fire-accidental',
       '118 - Trash or rubbish fire, contained', '442 - Overheated motor',
       '322 - Vehicle accident with injuries',
       '735 - Alarm system sounded due to malfunction',
       '554 - Assist invalid', '311 - Medical assist, assist EMS crew',
       '700 - False alarm or false call, other',
       '600 - Good intent call, other',
      

In [7]:
# create a df for internal fire.
internal_fire = ['412','210','522','520','113','411','444','653','445','730',\
                 '220','223','114','410','440','652','221','521','650','422',\
                 '213','671','200','441','443','423','420','431','251','371',\
                 '212','222','211','231','115','424','430','440','117','411',\
                 '531','412','210','651','445','200','413','114','451','443',\
                 '163','116','231','431']
df_internal_fire = df_fire[df_fire['Primary Situation'].str.contains('|'.join(internal_fire))]
df_internal_fire.shape

# Only focus on fire that are due to leakage, accidents, operation, etc.

412 - Gas leak (natural gas or LPG)
210 - Steam Rupture, steam, other
522 - Water or steam leak
520 - Water problem, other
113 - Cooking fire, confined to container
411 - Gasoline or other flammable liquid spill
413 - Oil or other combustible liquid spill
444 - Power line down (wire)
653 - Barbecue, tar kettle'
445 - Arcing, shorted electrical equipment
730 - System malfunction, other
220 - Rupture from air or gas, other
223 - Air/gas rupture of pressure or process
114 - Chimney/flue fire, confined to chimney
410 - Flammable gas or liquid condition, other
440 - Elec. wiring/equip. problem, other
652 - Steam/vapor/fog/dust mistaken for smoke
221 - Rupture air or gas, pipe/pipline
521 - Water evacuation
650 - Steam/gas mistaken for smoke, other
422 - Chemical spill or leak(haz.)
213 - Steam rupture, pressure/process vessel
671 - Hazmat release investigation w/no hazmat
200 - Overpressure rupture/explosion, overheat
441 - Heat- short circuit, defect/worn wiring
443 - Light ballast breakdown
423 - Refrigeration leak
420 - Toxic condition, other (haz.)
431 - Radiation leak, radioactive material
251 - Excess heat, scorth burns with no ign
371 - Electrocution or potential electrocution
212 - Overpressure rupture of steam boiler
222 - Rupture of boiler from air or gas
211 - Steam Rupture, Pipe or pipeline
231 - Chem. reaction rupture of process vessel
115 - Incinerator overload/mal.,fire confined
424 - Carbon monoxide incident21
430 - Radioactive condition, other
440 Electrical  wiring/equipment problem, other
117 Commercial Compactor fire, confined to rubbish
411 Gasoline or other flammable liquid spill
444 Power line down
531 Smoke or odor removal
412 Gas leak (natural gas or LPG)
210 Overpressure rupture from steam, other
113 Cooking fire, confined to container
651 Smoke scare, odor of smoke
653 Smoke from barbecue, tar kettle
445 Arcing, shorted electrical equipment
441 Heat from short circuit (wiring), defective/worn
423 Refrigeration leak
222 Overpressure rupture of boiler from air or gas
410 Combustible/flammable gas/liquid condition, other
424 Carbon monoxide incident
200 Overpressure rupture, explosion, overheat other
413 Oil or other combustible liquid spill
114 Chimney or flue fire, confined to chimney or flue
420 Toxic condition, other
220 Overpressure rupture from air or gas, other
451 Biological hazard, confirmed or suspected
443 Breakdown of light ballast
213 Steam rupture of pressure or process vessel
163 Outside gas or vapor combustion explosion
116 Fuel burner/boiler malfunction, fire confined
231 Chemical reaction rupture of process vessel
431 Radiation leak, radioactive material
430 Radioactive condition, other

# write function to process data
data assembly:
id, year, crime, occupancy, demographic, inspection, violation, fire incident last year, fire incident this year

df_parcel: gpd;

other df: pandas df

year: int

In [27]:
#parcel_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_parcel_lu/sf_parcel.geojson'
#df_parcel = gpd.read_file(parcel_file)
    
def process_fire(df_parcel, df_fire, year):
    
    # in order to spatial join, only need id and geometry to keep it simple
    df_parcel_simple = df_parcel[['objectid', 'geometry']]
    
    # create a buffer
    gdf_parcel_buffer = df_parcel_simple
    gdf_parcel_buffer.geometry = df_parcel_simple.geometry.buffer(0.0001) # unit is degree: 0.0003 is about 33 m or 100 feet
    
    # see if any of the incidents features is useful
    # if we are only counting incidents, then only incidents number,date and point are useful
    df_fire = df_fire[['Incident Number','Incident Date','point']]
    
    # select the year and previous year
    df_fire['Incident Date'] = pd.to_datetime(df_fire['Incident Date'])
    df_fire['year'] = df_fire['Incident Date'].dt.year
    # select this year and previous year
    df_fire = df_fire[(df_fire['year'] == year)|(df_fire['year'] == year-1)]
    
    df_fire['point'] = df_fire['point'].astype(str)
    df_fire = df_fire.dropna() # axis = 'point'

    # it seems that there is no null value, but instead, have a "nan" as a string
    df_fire = df_fire[df_fire['point'] != 'nan']
    df_fire['point'] = df_fire['point'].apply(wkt.loads)
    
    # change the fire df into geo df
    gdf_fire = gpd.GeoDataFrame(df_fire, geometry='point')

    # make sure the two dataset have the same projection
    gdf_fire.crs = df_parcel_simple.crs

    # divide the data into current year and previous year
    gdf_fire_current_year = gdf_fire[(gdf_fire['year'] == year)]
    gdf_fire_previous_year = gdf_fire[(gdf_fire['year'] == year-1)]
    
    ########################## current year
    # spatial join: using buffer
    fire_sjoin_current_year = gpd.sjoin(gdf_fire_current_year,gdf_parcel_buffer,how = 'left', op='within')

    # drop na
    fire_sjoin_current_year = fire_sjoin_current_year.dropna()
    # group by parcel id
    grouped_current_year = fire_sjoin_current_year.groupby('objectid').size()
    df_group_current_year = grouped_current_year.to_frame().reset_index()
    df_group_current_year.columns = ['objectid','fire_count']
    
    ###################### previous year
    # spatial join: using buffer
    fire_sjoin_previous_year = gpd.sjoin(gdf_fire_previous_year,gdf_parcel_buffer,how = 'left', op='within')

    # drop na
    fire_sjoin_previous_year = fire_sjoin_previous_year.dropna()
    # group by parcel id
    grouped_previous_year = fire_sjoin_previous_year.groupby('objectid').size()
    df_group_previous_year = grouped_previous_year.to_frame().reset_index()
    df_group_previous_year.columns = ['objectid','fire_count_last year']
    
    ######### join both
    # first convert gpd to pd
    parcel = pd.DataFrame(df_parcel.drop(columns='geometry'))

    # then merge using objectid
    merged_fire_data = parcel.merge(df_group_current_year, on='objectid', how='outer')
    merged_fire_data = merged_fire_data.merge(df_group_previous_year, on='objectid', how='outer')
    
    # replace nan with 0
    merged_fire_data['fire_count'] = merged_fire_data['fire_count'].fillna(0)
    merged_fire_data['fire_count_last year'] = merged_fire_data['fire_count_last year'].fillna(0)

    merged_fire_data['year'] = year 
    
    return merged_fire_data

# process inspection data

In [26]:
# select the types of inspection before inputing this function
def process_inspection(df_parcel, df_inspect, year):
    
    # in order to spatial join, only need id and geometry to keep it simple
    df_parcel_simple = df_parcel[['objectid', 'geometry']]
    
    # select 
    
    df_inspect['Inspection Start Date'] = pd.to_datetime(df_inspect['Inspection Start Date'])
    df_inspect['year'] = df_inspect['Inspection Start Date'].dt.year
    df_inspect = df_inspect[(df_inspect['year'] == year)]
    
    # this need to drop na in geometry fileds, otherwise it would fail
    df_inspect = df_inspect.dropna(subset=['point'])
    df_inspect['point'] = df_inspect.apply(lambda row: shape(json.loads(row['point'].replace("\'", "\""))), axis=1)
    gdf_inspect = gpd.GeoDataFrame(df_inspect, geometry='point')

    # make sure the two dataset have the same projection
    gdf_inspect.crs = df_parcel_simple.crs
    
    inspect_sjoin = gpd.sjoin(gdf_inspect, df_parcel_simple, how="inner", op='within')
    
    # dropnas
    inspect_sjoin = inspect_sjoin.dropna()
    
    # group by parcel id
    grouped = inspect_sjoin.groupby('objectid').size()
    df_group = grouped.to_frame().reset_index()
    df_group.columns = ['objectid','complaints_count']
    
    ##### join with parcel
    # first convert gpd to pd
    #parcel = pd.DataFrame(df_parcel.drop(columns='geometry'))

    # then merge using objectid
    #merged_inspect_data = parcel.merge(df_group, on='objectid', how='outer')
    
    # replace nan with 0
    #merged_inspect_data['complaints_count'] = merged_inspect_data['complaints_count'].fillna(0)

    return df_group

# process violation data

In [25]:
def process_violation(df_parcel, df_violate, year):
    
    # in order to spatial join, only need id and geometry to keep it simple
    df_parcel_simple = df_parcel[['objectid', 'geometry']]
    
    # select year 
    df_violate['violation date'] = pd.to_datetime(df_violate['violation date'])
    df_violate['year'] = df_violate['violation date'].dt.year
    df_violate = df_violate[(df_violate['year'] == year)]
    
    # this need to drop na in geometry fileds, otherwise it would fail
    df_violate = df_violate.dropna(subset=['Location'])
    df_violate['Location'] = df_violate.apply(lambda row: shape(json.loads(row['Location'].replace("\'", "\""))), axis=1)
    gdf_violate = gpd.GeoDataFrame(df_violate, geometry='Location')

    # make sure the two dataset have the same projection
    gdf_violate.crs = df_parcel_simple.crs
    
    violate_sjoin = gpd.sjoin(gdf_violate, df_parcel_simple, how="inner", op='within')
    
    # dropnas
    violate_sjoin = violate_sjoin.dropna()
    
    # group by parcel id
    grouped = violate_sjoin.groupby('objectid').size()
    df_group = grouped.to_frame().reset_index()
    df_group.columns = ['objectid','violation_count']
    
    ##### join with parcel
    # first convert gpd to pd
    #parcel = pd.DataFrame(df_parcel.drop(columns='geometry'))

    # then merge using objectid
    #merged_violation_data = parcel.merge(df_group, on='objectid', how='outer')
    
    # replace nan with 0
    #merged_violation_data['violation_count'] = merged_violation_data['violation_count'].fillna(0)
    
    
    
    return df_group

# process building permit data

In [28]:
def process_permit(df_parcel, df_permit, year):
    
    # in order to spatial join, only need id and geometry to keep it simple
    df_parcel_simple = df_parcel[['objectid', 'geometry']]
    
    # select year 
    df_permit['Issued Date'] = pd.to_datetime(df_permit['Issued Date'])
    df_permit['year'] = df_permit['Issued Date'].dt.year
    df_permit = df_permit[(df_permit['year'] == year)]
    
    # this need to drop na in geometry fileds, otherwise it would fail
    df_permit = df_permit.dropna(subset=['lat','long'])
    gdf_permit = gpd.GeoDataFrame(df_permit, geometry=gpd.points_from_xy(df_permit.long, df_permit.lat))

    # make sure the two dataset have the same projection
    gdf_permit.crs = df_parcel_simple.crs
    
    permit_sjoin = gpd.sjoin(gdf_permit, df_parcel_simple, how="inner", op='within')
    
    # dropnas
    permit_sjoin = permit_sjoin.dropna()
    
    # group by parcel id
    grouped = permit_sjoin.groupby('objectid').size()
    df_group = grouped.to_frame().reset_index()
    df_group.columns = ['objectid','building_permit']
    

    return df_group

# assemble the whole dataset

In [22]:
# load data

###################### Fire data
parcel_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_parcel_lu/sf_parcel.geojson'
df_parcel = gpd.read_file(parcel_file)

fire_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/Fire_Incidents_SF.csv'
df_fire = pd.read_csv(fire_file)

# based on others
structure_fire = ['111','112']

df_structure_fire = df_fire[df_fire['Primary Situation'].str.contains('|'.join(structure_fire))]

# create a df for internal fire.
internal_fire = ['412','210','522','520','113','411','444','653','445','730',\
                 '220','223','114','410','440','652','221','521','650','422',\
                 '213','671','200','441','443','423','420','431','251','371',\
                 '212','222','211','231','115','424','430','440','117','411',\
                 '531','412','210','651','445','200','413','114','451','443',\
                 '163','116','231','431']
df_internal_fire = df_fire[df_fire['Primary Situation'].str.contains('|'.join(internal_fire))]
df_internal_fire.shape

######################### inspection data
inspection_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_fire_data/fire-inspections.csv'
df_inspect = pd.read_csv(inspection_file)

#  'Complaint Inspection'
df_inspect = df_inspect[df_inspect['Inspection Type Description'] == 'Complaint Inspection']

#################### violation data
violation_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_fire_data/fire-violations.csv'
df_violate = pd.read_csv(violation_file)

################ permit data
permit_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_building_permit/Building_Permits.csv'
df_permit = pd.read_csv(permit_file)

df_permit['Location'] = df_permit['Location'].str.strip('()')
df_permit[['lat','long']] = df_permit['Location'].str.split(', ',expand=True)

C:\Users\Ruoying\anaconda3\envs\geoenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (10,12,13,20,38,45,46,55,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Ruoying\anaconda3\envs\geoenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1,7,11,15,16,21,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Ruoying\anaconda3\envs\geoenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [82]:
############################################# don't need to run this again
# check the year of each dataset
# seems that 2006 is the earliest for violation data
df_fire['Incident Date'] = pd.to_datetime(df_fire['Incident Date'])
df_fire['year'] = df_fire['Incident Date'].dt.year
print(df_fire['year'].unique())

df_inspect['Inspection Start Date'] = pd.to_datetime(df_inspect['Inspection Start Date'])
df_inspect['year'] = df_inspect['Inspection Start Date'].dt.year
print(df_inspect['year'].unique())

df_violate['violation date'] = pd.to_datetime(df_violate['violation date'])
df_violate['year'] = df_violate['violation date'].dt.year
print(df_violate['year'].unique())

[2008 2003 2004 2007 2005 2006 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020]
[2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010 2009 2008 2007
 2006 2005 2004]
[2019 2018 2016 2014 2009 2020 2017 2008 2006 2011 2010 2007 2012 2015
 2005 2013]


In [29]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# initiate df
final_df = pd.DataFrame()

# so start from 2007, up to 2019
# permit data only 2013-2018

for year in range(2012,2020):
    
    building_fire = process_fire(df_parcel, df_internal_fire, year)

    inspect_result = process_inspection(df_parcel, df_inspect, year)
    violate_result = process_violation(df_parcel, df_violate, year)
    permit_result = process_permit(df_parcel, df_permit, year)
    
    # merge inspection, violation data to building/fire
    building_fire = building_fire.merge(inspect_result,on='objectid', how='outer')
    building_fire = building_fire.merge(violate_result,on='objectid', how='outer')
    building_fire = building_fire.merge(permit_result,on='objectid', how='outer')
    
    # replace nan with 0
    building_fire['complaints_count'] = building_fire['complaints_count'].fillna(0)
    building_fire['violation_count'] = building_fire['violation_count'].fillna(0)
    building_fire['building_permit'] = building_fire['building_permit'].fillna(0)
    
    final_df = pd.concat([final_df,building_fire])
    
    print(year, end =", ")


2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 

In [30]:
final_df.shape

(1243744, 28)

In [31]:
final_df.columns

Index(['yrbuilt', 'from_st', 'total_uses', 'shape_area', 'cie', 'landuse',
       'mips', 'mapblklot', 'objectid', 'shape_leng', 'visitor', 'st_type',
       'bldgsqft', 'resunits', 'pdr', 'street', 'retail', 'to_st', 'block_num',
       'blklot', 'lot_num', 'med', 'fire_count', 'fire_count_last year',
       'year', 'complaints_count', 'violation_count', 'building_permit'],
      dtype='object')

In [32]:
final_df.to_csv('final_analysis.csv', index=False)

# lastly: join parcel with block

In [33]:
parcel_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/SF_parcel_lu/sf_parcel.geojson'
df_parcel = gpd.read_file(parcel_file)

block_file = 'D:/Study/insight/project/fire_risk_prediction_SF/data/Census 2010_ Blocks for San Francisco.geojson'
df_block = gpd.read_file(block_file)

In [35]:
df_parcel_simple = df_parcel[['objectid', 'geometry']]

df_block['block_id'] = pd.to_numeric(df_block.geoid10, errors='coerce').astype(np.int64)
df_block_simple = df_block[['block_id', 'geometry']]

df_block_simple.crs = df_parcel_simple.crs

parcel_sjoin = gpd.sjoin(df_parcel_simple, df_block_simple, how="inner", op='intersects')

In [36]:
parcel_sjoin.head()

,objectid,geometry,index_right,block_id
0,1,"MULTIPOLYGON (((-122.42108 37.80478, -122.4211...",1999,60750102001011
1,2,"MULTIPOLYGON (((-122.42118 37.80476, -122.4212...",1999,60750102001011
2,3,"MULTIPOLYGON (((-122.42126 37.80475, -122.4213...",1999,60750102001011
3,4,"MULTIPOLYGON (((-122.42135 37.80474, -122.4215...",1999,60750102001011
4,5,"MULTIPOLYGON (((-122.42153 37.80472, -122.4217...",1999,60750102001011


In [39]:
parcel_sjoin.shape

(158633, 2)

In [38]:
parcel_sjoin = parcel_sjoin[['objectid','block_id']]
parcel_sjoin.to_csv('parcel_blk.csv', index=False)